In [3]:
import pandas as pd

In [24]:
dfs = [] 
for i in range(1, 6): 
    filename = f"./data/output/val_v14-small_fold_{i}.json"
    print(filename)
    dfs.append(pd.read_json(filename))

./data/output/val_v14-small_fold_1.json
./data/output/val_v14-small_fold_2.json
./data/output/val_v14-small_fold_3.json
./data/output/val_v14-small_fold_4.json
./data/output/val_v14-small_fold_5.json


In [25]:
df = pd.concat(dfs).reset_index(drop=True)
# df = df.drop(columns=["tag_title", "checklists"])

In [26]:
df = df.drop(columns=["Original_Tag", "Predicted_Tag", "Domain"])

In [27]:
df = df.rename(columns={"tags": "Original_Tag", "predicted_tags": "Predicted_Tag"})

In [28]:
df.section_content.unique().shape

(6785,)

In [29]:
df = df[df.filename.str.contains('synth') == False]

In [30]:
print(df.shape)

(6785, 7)


In [31]:
df.columns

Index(['filename', 'title', 'section_number', 'section_content', 'match',
       'Original_Tag', 'Predicted_Tag'],
      dtype='object')

In [32]:
df.head()

,filename,title,section_number,section_content,match,Original_Tag,Predicted_Tag
0,"Paramount Global - $3,500,000,000 AMENDED AND ...",Terms Generally,SECTION 1.2,Title is Terms Generally\nSection 1.2. Terms G...,True,NA,NA
1,"Paramount Global - $3,500,000,000 AMENDED AND ...",Currency Equivalents,SECTION 1.3,Title is Currency Equivalents\nSection 1.3. Cu...,True,NA,NA
2,"Paramount Global - $3,500,000,000 AMENDED AND ...",Commitments,SECTION 2.1,Title is Commitments\nSection 2.1. Commitments...,True,Facilities / Instrument,Facilities / Instrument
3,"Paramount Global - $3,500,000,000 AMENDED AND ...",Revolving Credit Loans; Competitive Loans,SECTION 2.2,Title is Revolving Credit Loans; Competitive L...,True,Facilities / Instrument,Facilities / Instrument
4,"Paramount Global - $3,500,000,000 AMENDED AND ...",Competitive Bid Procedure,SECTION 2.3,Title is Competitive Bid Procedure\nSection 2....,True,NA,NA


In [33]:
import re

In [34]:
def extract_section_num(x): 
    text = x.split("\n")[1].strip().upper()
    matches = re.findall("^SECTION\s+\d+\.\d{1,2}", text)
    if matches:
        return matches[0]
    matches  = re.findall("^SECTION\s+\d+\.?\d{1,2}", text)
    if matches:
        return matches[0]
    matches  = re.findall("^\d+\.?\d{1,2}", text, flags=re.IGNORECASE)
    if matches:
        return matches[0]
    return "NOT FOUND"

In [35]:
df['section_number'] = df.section_content.apply(extract_section_num)

In [36]:
df[df.section_number == "NOT FOUND"].shape

(0, 7)

In [37]:
df['match'] = df['Original_Tag'] == df["Predicted_Tag"]

In [38]:
df = df[['filename', 'title', 'section_number', 'section_content', 'Original_Tag', 'Predicted_Tag', "match"]]

In [39]:
df.shape

(6785, 7)

In [40]:
df.head()

,filename,title,section_number,section_content,Original_Tag,Predicted_Tag,match
0,"Paramount Global - $3,500,000,000 AMENDED AND ...",Terms Generally,SECTION 1.2,Title is Terms Generally\nSection 1.2. Terms G...,NA,NA,True
1,"Paramount Global - $3,500,000,000 AMENDED AND ...",Currency Equivalents,SECTION 1.3,Title is Currency Equivalents\nSection 1.3. Cu...,NA,NA,True
2,"Paramount Global - $3,500,000,000 AMENDED AND ...",Commitments,SECTION 2.1,Title is Commitments\nSection 2.1. Commitments...,Facilities / Instrument,Facilities / Instrument,True
3,"Paramount Global - $3,500,000,000 AMENDED AND ...",Revolving Credit Loans; Competitive Loans,SECTION 2.2,Title is Revolving Credit Loans; Competitive L...,Facilities / Instrument,Facilities / Instrument,True
4,"Paramount Global - $3,500,000,000 AMENDED AND ...",Competitive Bid Procedure,SECTION 2.3,Title is Competitive Bid Procedure\nSection 2....,NA,NA,True


In [42]:
df.to_excel("./data/output/predictions_v14_kfold_small.xlsx", index=False)

In [169]:
df.shape

(6789, 6)

In [14]:
df[df.Original_Tag != df.Predicted_Tag].shape

(320, 7)